In [6]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

In [7]:
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="6793e723-756c-4c5d-84c0-812f1bb4c679", # Subscription ID
    resource_group_name="JuvlinResourceGroup",              # Resource Group
    workspace_name="JuvlinWorkspace",
)

In [8]:
# Register the environment:

from azure.ai.ml.entities import Environment

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    # conda_file="../../model-1/environment/conda1.yaml",
    conda_file="env/conda1.yaml",
    name="my-env",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(env_docker_conda)

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'my-env', 'description': 'Environment created from a Docker image plus Conda environment.', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/6793e723-756c-4c5d-84c0-812f1bb4c679/resourceGroups/JuvlinResourceGroup/providers/Microsoft.MachineLearningServices/workspaces/JuvlinWorkspace/environments/my-env/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/pinhe512611/code/Users/pinhe51261/Code', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7fa196322260>, 'serialize': <msrest.serialization.Serializer object at 0x7fa196322470>, 'version': '1', 'conda_file': {'channels': ['conda-forge'], 'dependencies

In [9]:
# Register the Model

from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

file_model = Model(
    path="milestone1_model/milestone1_best_model/",
    type=AssetTypes.CUSTOM_MODEL,
    name="my-model",
    description="Model created from local file.",
)
ml_client.models.create_or_update(file_model)

Uploading milestone1_best_model (0.02 MBs): 100%|██████████| 18574/18574 [00:00<00:00, 562426.02it/s]




Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'my-model', 'description': 'Model created from local file.', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/6793e723-756c-4c5d-84c0-812f1bb4c679/resourceGroups/JuvlinResourceGroup/providers/Microsoft.MachineLearningServices/workspaces/JuvlinWorkspace/models/my-model/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/pinhe512611/code/Users/pinhe51261/Code', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7fa19415c190>, 'serialize': <msrest.serialization.Serializer object at 0x7fa19416bc10>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/6793e723-756c-4c5d-84c0-812f1bb4c679/resourceGroups/JuvlinResourceGroup/workspaces/JuvlinWorkspace/datastores/workspaceblobstore/paths/LocalUpload/349027d1a5359bbddb5729bf

In [18]:
# Configure the endpoint:

from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.ai.ml.constants import AssetTypes

endpoint_name = "endpoint-juvlin"

endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,   # Unique Endpoint Name within Deployment
    description="An online endpoint serving an MLflow model, for Milestone1 Advertising Prediction task",
                          # A string describing the purpose of the endpoint
    auth_mode="key",      # Authentication mode for endpoint - API key
    tags={"foo": "bar"},  # Dictionary of key-value pairs to tag the Endpoint
)

In [19]:
# Create the endpoint:

ml_client.online_endpoints.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-juvlin.eastus2.inference.ml.azure.com/score', 'openapi_uri': 'https://endpoint-juvlin.eastus2.inference.ml.azure.com/swagger.json', 'name': 'endpoint-juvlin', 'description': 'An online endpoint serving an MLflow model, for Milestone1 Advertising Prediction task', 'tags': {'foo': 'bar'}, 'properties': {'createdBy': 'Juvlin Pinheiro', 'createdAt': '2024-11-14T00:19:53.555767+0000', 'lastModifiedAt': '2024-11-14T00:19:53.555767+0000', 'azureml.onlineendpointid': '/subscriptions/6793e723-756c-4c5d-84c0-812f1bb4c679/resourcegroups/juvlinresourcegroup/providers/microsoft.machinelearningservices/workspaces/juvlinworkspace/onlineendpoints/endpoint-juvlin', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/6793e723-756c-4c5d-84c0-812f1bb4c679/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/oeidp:533a1434-a3da-4be8-bf

In [22]:
# Configure Model deployment, use the registered model and environment:

model = "azureml:my-model:1"
env = "azureml:my-env:1"
 
blue_deployment_with_registered_assets = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="src", scoring_script="score1.py"
    ),
    instance_type="STANDARD_E8S_V3",
    instance_count=1,
)

In [23]:
# Create the deployment:

ml_client.online_deployments.begin_create_or_update(blue_deployment_with_registered_assets)

Check: endpoint endpoint-juvlin exists


.......................

In [24]:
# Check the endpoint's status to see whether the model was deployed without error:

ml_client.online_endpoints.get(name=endpoint_name)

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-juvlin.eastus2.inference.ml.azure.com/score', 'openapi_uri': 'https://endpoint-juvlin.eastus2.inference.ml.azure.com/swagger.json', 'name': 'endpoint-juvlin', 'description': 'An online endpoint serving an MLflow model, for Milestone1 Advertising Prediction task', 'tags': {'foo': 'bar'}, 'properties': {'createdBy': 'Juvlin Pinheiro', 'createdAt': '2024-11-14T00:19:53.555767+0000', 'lastModifiedAt': '2024-11-14T00:19:53.555767+0000', 'azureml.onlineendpointid': '/subscriptions/6793e723-756c-4c5d-84c0-812f1bb4c679/resourcegroups/juvlinresourcegroup/providers/microsoft.machinelearningservices/workspaces/juvlinworkspace/onlineendpoints/endpoint-juvlin', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/6793e723-756c-4c5d-84c0-812f1bb4c679/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/oeidp:533a1434-a3da-4be8-bf

In [25]:
# List all the endpoints in the workspace in a table format by using the list method:

for endpoint in ml_client.online_endpoints.list():
    print(endpoint.name)

milestone1-endpoint-juvlin
endpoint-juvlin


In [26]:
# View log output by using the get_logs method:

ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=endpoint_name, lines=50
)

'There are no logs for this deployment at the moment. Please try again later.'

In [30]:
# Send a sample request using a json file.

# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name="blue",
    request_file="../src_test/sample-request-sklearn1.json",
)

ServiceResponseError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))